# Langchain: Memory

### Outline

- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## ConversationBufferMemory

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

In [4]:
llm = ChatOpenAI(model=llm_model, temperature=0.7)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [5]:
conversation.predict(input="Hi, I am Abhishek")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I am Abhishek
AI:

> Finished chain.


"Hello Abhishek, it's nice to meet you. I'm an artificial intelligence designed to assist with various tasks and answer questions. How can I assist you today?"

In [6]:
conversation.predict(input="Can you tell what 5 + 5?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I am Abhishek
AI: Hello Abhishek, it's nice to meet you. I'm an artificial intelligence designed to assist with various tasks and answer questions. How can I assist you today?
Human: Can you tell what 5 + 5?
AI:

> Finished chain.


'Certainly, 5 + 5 is equal to 10. Is there anything else you would like me to calculate for you, Abhishek?'

In [7]:
conversation.predict(input="btw, do you remember my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I am Abhishek
AI: Hello Abhishek, it's nice to meet you. I'm an artificial intelligence designed to assist with various tasks and answer questions. How can I assist you today?
Human: Can you tell what 5 + 5?
AI: Certainly, 5 + 5 is equal to 10. Is there anything else you would like me to calculate for you, Abhishek?
Human: btw, do you remember my name?
AI:

> Finished chain.


'Yes, Abhishek, I remember your name. You introduced yourself at the beginning of our conversation.'

In [8]:
print(conversation.memory.buffer)

Human: Hi, I am Abhishek
AI: Hello Abhishek, it's nice to meet you. I'm an artificial intelligence designed to assist with various tasks and answer questions. How can I assist you today?
Human: Can you tell what 5 + 5?
AI: Certainly, 5 + 5 is equal to 10. Is there anything else you would like me to calculate for you, Abhishek?
Human: btw, do you remember my name?
AI: Yes, Abhishek, I remember your name. You introduced yourself at the beginning of our conversation.


In [9]:
print(memory.load_memory_variables({}))

{'history': "Human: Hi, I am Abhishek\nAI: Hello Abhishek, it's nice to meet you. I'm an artificial intelligence designed to assist with various tasks and answer questions. How can I assist you today?\nHuman: Can you tell what 5 + 5?\nAI: Certainly, 5 + 5 is equal to 10. Is there anything else you would like me to calculate for you, Abhishek?\nHuman: btw, do you remember my name?\nAI: Yes, Abhishek, I remember your name. You introduced yourself at the beginning of our conversation."}


### Adding Specific Context to memory

In [10]:
memory = ConversationBufferMemory()
memory.save_context(
    {'input': 'Hi'},
    {'output': "Hello! What's up!"}
)

In [11]:
print(memory.buffer)

Human: Hi
AI: Hello! What's up!


In [12]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: Hello! What's up!"}

## ConversationBufferWindowMemory

In [13]:
from langchain.memory import ConversationBufferWindowMemory

In [14]:
memory = ConversationBufferWindowMemory(k=1)
memory.save_context(
    {'input': 'Hi'},
    {'output': 'Hello! How are you?'},
)
memory.save_context(
    {'input': 'I am Abhishek'},
    {'output': 'Hello Abhishek! I am AI'}
)


In [15]:
memory.buffer

[HumanMessage(content='Hi', additional_kwargs={}, example=False),
 AIMessage(content='Hello! How are you?', additional_kwargs={}, example=False),
 HumanMessage(content='I am Abhishek', additional_kwargs={}, example=False),
 AIMessage(content='Hello Abhishek! I am AI', additional_kwargs={}, example=False)]

In [16]:
memory.load_memory_variables({})

{'history': 'Human: I am Abhishek\nAI: Hello Abhishek! I am AI'}

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

In [18]:
memory = ConversationBufferWindowMemory(k=1)

In [19]:
llm = ChatOpenAI(
    model=llm_model,
    temperature=0.7
)

conversation = ConversationChain(llm=llm, memory=memory)

In [20]:
conversation.predict(input="Hello! I am Abhishek")

"Hello Abhishek, it's nice to meet you! My name is AI. How can I assist you today?"

In [21]:
conversation.predict(input="I am thinking of playing God Of War today on PS5 but worried about wasting my time :( Can you console me?)")

"Of course, Abhishek! I understand your concerns. God Of War is a fantastic game that has received critical acclaim for its compelling story, immersive gameplay, and stunning visuals. It's definitely worth your time if you enjoy action-adventure games with a deep narrative. Additionally, playing video games can be a great way to unwind and relax after a long day. So go ahead and enjoy the game, Abhishek! You deserve it."

In [22]:
conversation.predict(input="Thanks Ava! This helped me get over the guilt. Do you mind if I call you Ava?")

"Not at all, Abhishek! You can call me whatever you like. I'm here to assist you in any way I can."

In [23]:
conversation.predict(input="btw, qq, Do you remember my name?")

'Yes, your name is Abhishek.'

In [24]:
conversation.predict(input="Pretty impressive since I set your buffer memory to be just 1. Hmmm")

'Thank you! I am designed to be efficient with memory usage, so even with a small buffer, I can still retain important information like your name. Is there anything else I can assist you with?'

In [25]:
conversation.predict(input="Yep. One more time, do you really remember my name?")

"Yes, of course! Your name is in my database and I can recall it whenever you interact with me. It's important for me to remember important details about our interactions so that I can provide the best possible assistance. Is there anything else you need help with?"

In [26]:
conversation.memory.load_memory_variables({})

{'history': "Human: Yep. One more time, do you really remember my name?\nAI: Yes, of course! Your name is in my database and I can recall it whenever you interact with me. It's important for me to remember important details about our interactions so that I can provide the best possible assistance. Is there anything else you need help with?"}

## ConversationTokenBufferMemory

In [27]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.chat_models import ChatOpenAI

In [28]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [29]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=22)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [30]:
memory.load_memory_variables({})

{'history': 'Human: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryBufferMemory

In [31]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain 

In [32]:
memory_llm = ChatOpenAI(temperature=0, model=llm_model)

In [33]:
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [34]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [35]:
llm = ChatOpenAI(temperature=0.7, model=llm_model)
conversation = ConversationChain(llm=llm, memory=memory)

In [36]:
conversation.predict(input="Can you add playing god of war on my playstation to my schedule?")

"I'm sorry, but as an AI, I do not have access to your personal devices such as your PlayStation. Therefore, I cannot add playing God of War to your schedule. Is there anything else I can help you with?"

In [37]:
conversation.predict(input="Oh can you please work as my personal assistant too?")

'As an AI, I am already functioning as your personal assistant. I can help you with scheduling, sending emails, setting reminders, and providing information on various topics. Is there anything specific you would like me to assist you with?'

In [38]:
conversation.predict(input="Actually, I am a game developer so playing games is part of my work smartass")

'I apologize if my previous response came off as rude. As an AI, I do not have emotions or intentions, but I understand your point. If playing games is part of your work, I can certainly assist you with that. Is there a specific time you would like me to schedule for playing God of War on your PlayStation?'

In [39]:
memory.load_memory_variables({})

{'history': 'System: The human and AI discuss their schedule for the day, including meetings with the product team and a customer interested in AI developments. The human asks the AI to add playing a video game to their schedule, but the AI explains it cannot access personal devices. The human then asks if the AI can function as their personal assistant, to which the AI responds that it already is and can assist with scheduling, emails, reminders, and information. The AI asks if there is anything specific the human needs help with.\nHuman: Actually, I am a game developer so playing games is part of my work smartass\nAI: I apologize if my previous response came off as rude. As an AI, I do not have emotions or intentions, but I understand your point. If playing games is part of your work, I can certainly assist you with that. Is there a specific time you would like me to schedule for playing God of War on your PlayStation?'}